In [33]:
import pymongo
import pandas as pd
import itertools
import csv
import tqdm

OUTFILE = 'graduates.csv'

In [5]:
client = pymongo.MongoClient()
db = client['uni']

In [37]:
def get_graduates(vpo1_collection):
    return vpo1_collection.aggregate(
        pipeline=[
        {
            '$match': {
                'section': '2.1.3'
            }
        }, {
            '$unwind': {
                'path': '$data', 
                'preserveNullAndEmptyArrays': False
            }
        }, {
            '$match': {
                'data.name': {
                    '$regex': 'Выпуск фактический.*'
                }
            }
        }, {
            '$unwind': {
                'path': '$data.records', 
                'preserveNullAndEmptyArrays': False
            }
        }, {
            '$project': {
                'region': 1, 
                'funded_by': 1, 
                'time_involvement': 1,  
                'colname': '$data.name', 
                'group': '$data.records.group', 
                'program': '$data.records.name', 
                'value': '$data.records.value'
            }
        }
    ])

In [55]:
fieldnames = list(next(get_graduates(db.vpo1)).keys())
fieldnames.remove('_id')
fieldnames.remove('group')
fieldnames

['region', 'funded_by', 'time_involvement', 'colname', 'program', 'value']

In [56]:
with open(OUTFILE, 'w', newline='') as f:
    f.truncate(0)
    writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    for record in tqdm.tqdm(get_graduates(db.vpo1)):
        record['program'] = '{}. {}'.format(record['group'], record['program'])
        for k in list(record.keys()):
            if k not in fieldnames:
                del record[k]
        writer.writerow(record)

56478it [00:02, 27601.56it/s]


In [57]:
df = pd.read_csv(OUTFILE)
df.head()

,region,funded_by,time_involvement,colname,program,value
0,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Программы бакалавриата - всего,6940.0
1,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,25.0
2,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Прикладная математика и информатика,79.0
3,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Механика и математическое моделирование,18.0
4,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Прикладная математика,14.0


In [60]:
df[df['program'] == '1. Математика'].head()

,region,funded_by,time_involvement,colname,program,value
1,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,25.0
236,Воронежская область,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,1.0
889,Российская Федерация,Всего,очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,463.0
1403,Российская Федерация,Всего,очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,4.0
3325,Российская Федерация,Всего,очно-заочная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,18.0


In [59]:
df['colname'].unique()

array(['Выпуск фактический с 01.10. прошлого года по 30.09. текущего года всего (сумма гр. 10, 12, 13, 15)',
       'Выпуск фактический с 01.10. прошлого года по 30.09. текущего года из них лица с ОВЗ, инвалиды, детиинвалиды'],
      dtype=object)